In [13]:
import heapq
import pandas as pd
import random
from datetime import datetime, timedelta
from collections import deque

# --------------------------------------------------------------------------
# DATA CLEANING FUNCTIONS
# --------------------------------------------------------------------------
def remove_outliers_iqr(df, cols):
    for c in cols:
        Q1, Q3 = df[c].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        df = df[df[c].between(Q1 - 2 * IQR, Q3 + 2 * IQR)]
    return df

def drop_rows_without_true_outfeed(df, prefix="Outfeed"):
    cols = [c for c in df.columns if c.startswith(prefix)]
    return df[df[cols].any(axis=1)] if cols else df

def clean_parcel_data(df):
    df = df.dropna().reset_index(drop=True)
    df = remove_outliers_iqr(df, ["Length", "Width", "Height"])
    df = drop_rows_without_true_outfeed(df)
    return df

def load_parcels_from_clean_df(df):
    parcels = []
    # 1) Find every column that starts with "Outfeed " and sort by the numeric suffix
    outfeed_columns = sorted(
        [col for col in df.columns if col.startswith("Outfeed ")],
        key=lambda c: int(c.split(" ")[1])
    )
    # 2) Now 'num_outfeeds' is simply how many of those columns exist
    num_outfeeds = len(outfeed_columns)

    # 3) For each row, record the parcel attributes + any outfeed columns that are True/1
    for _, r in df.iterrows():
        # Build a list of indices [0..num_outfeeds-1] where that column is truthy
        feasible = [
            idx
            for idx, col_name in enumerate(outfeed_columns)
            if bool(r[col_name])
        ]
        parcels.append(
            Parcel(
                pid=int(r["Parcel Number"]),
                arrival_time=pd.to_datetime(r["Arrival Time"]),
                length=float(r["Length"]),
                width=float(r["Width"]),
                height=float(r["Height"]),
                weight=float(r["Weight"]),
                feasible=feasible
            )
        )

    # 4) Return the parcels sorted by arrival_time, plus how many outfeeds we found
    return sorted(parcels, key=lambda p: p.arrival_time), num_outfeeds

# --------------------------------------------------------------------------
# EVENT, FES, PARCEL
# --------------------------------------------------------------------------
class Event:
    ARRIVAL = 0
    ENTER_SCANNER = 1
    ENTER_OUTFEED = 2
    EXIT_OUTFEED = 3
    RECIRCULATE = 4
    def __init__(self, typ, time, parcel, outfeed_id=None):
        self.type = typ
        self.time = time
        self.parcel = parcel
        self.outfeed_id = outfeed_id
    def __lt__(self, other):
        return self.time < other.time

class FES:
    def __init__(self): self.events = []
    def add(self, e): heapq.heappush(self.events, e)
    def next(self): return heapq.heappop(self.events)
    def isEmpty(self): return not self.events

class Parcel:
    def __init__(self, pid, arrival_time, length, width, height, weight, feasible):
        self.id = pid
        self.arrival_time = arrival_time
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.feasible_outfeeds = feasible
        self.recirculation_count = 0
    def get_volume(self): return self.length * self.width * self.height

# --------------------------------------------------------------------------
# OUTFEED MODEL
# --------------------------------------------------------------------------
def compute_outfeed_time(parcel):
    base_time = 4.5
    volume = parcel.get_volume()
    if volume < 0.035:
        volume_class_delay = 0
    elif volume < 0.055:
        volume_class_delay = 1
    else:
        volume_class_delay = 2
    weight = parcel.weight
    if weight < 1700:
        weight_class_delay = 0
    elif weight < 2800:
        weight_class_delay = 1
    else:
        weight_class_delay = 2
    return base_time + volume_class_delay + weight_class_delay

class Outfeed:
    def __init__(self, max_length=4.5):
        self.max_length = max_length
        self.current_length = 0.0
        self.queue = []   # list of (Parcel, service_time)
        self.next_time = 0.0
    def can_accept(self, parcel):
        return self.current_length + parcel.length <= self.max_length
    def add_parcel(self, parcel):
        service_t = compute_outfeed_time(parcel)
        self.queue.append((parcel, service_t))
        self.current_length += parcel.length
        if len(self.queue) == 1:
            self.next_time = service_t
        return service_t
    def update(self, dt):
        self.next_time -= dt
        if self.next_time <= 0 and self.queue:
            p, _ = self.queue.pop(0)
            self.current_length -= p.length
            if self.queue:
                self.next_time = self.queue[0][1]

# --------------------------------------------------------------------------
# POSISORTER WITH TIME-BASED LOAD BALANCING
# --------------------------------------------------------------------------
class PosiSorterSystem:
    REBALANCE_INTERVAL = 100  # (you already set this high)
    
    def __init__(self, layout_df, num_outfeeds):
        L = layout_df.set_index("Layout property")["Value"]
        self.belt_speed = L["Belt Speed"]
        self.d_in_sc = L["Distance Infeeds to Scanner"]
        self.d_sc_of = L["Distance Scanner to Outfeeds"]
        self.d_between = L["Distance between Outfeeds"]
        self.d_of_in = L["Distance Outfeeds to Infeeds"]
        self.num_outfeeds = num_outfeeds
        
        # Create each outfeed with the given max_length
        self.outfeeds = [Outfeed() for _ in range(self.num_outfeeds)]
        
        # Statistics
        self.recirculated_count = 0
        self.outfeed_counts = [0] * self.num_outfeeds
        self.first_pass_failures = set()
        
        # Load‐balancing structures
        self.loads = {k: 0.0 for k in range(self.num_outfeeds)}
        self.service_times = {}   # parcel.id → service_time
        self.assignment = {}      # parcel.id → outfeed index (or None)
        
        # Sliding window to hold (arrival_to_scanner_time, Parcel)
        self.WINDOW_DURATION = self.d_sc_of / self.belt_speed
        self.window = deque()
        self.rebal_ctr = 0

    def greedy(self, p):
        feas = [k for k in p.feasible_outfeeds if self.outfeeds[k].can_accept(p)]
        if not feas:
            return None
        return min(feas, key=lambda k: self.loads[k])

    def imbalance(self, loads):
        return max(loads.values()) - min(loads.values())

    def run_local_search(self, max_iters=100):
        loads = self.loads.copy()
        assign_w = {}
        # 1) Greedy‐initialize assignments within the current window
        for _, p in self.window:
            k = self.greedy(p)
            assign_w[p.id] = k
            if k is not None:
                loads[k] += self.service_times.get(p.id, 0)
        # 2) Hill‐climb to reduce max(loads)-min(loads)
        for _ in range(max_iters):
            improved = False
            for _, p in self.window:
                cur = assign_w[p.id]
                for k in p.feasible_outfeeds:
                    if k == cur or not self.outfeeds[k].can_accept(p):
                        continue
                    st = self.service_times.get(p.id, 0)
                    new_loads = loads.copy()
                    if cur is not None:
                        new_loads[cur] -= st
                    new_loads[k] += st
                    if self.imbalance(new_loads) < self.imbalance(loads):
                        loads, assign_w[p.id] = new_loads, k
                        improved = True
                        break
                if improved:
                    break
            if not improved:
                break
        # 3) Write the (possibly rebalanced) assignments back
        for pid, k in assign_w.items():
            self.assignment[pid] = k

    def handle_enter_scanner(self, evt, fes):
        p = evt.parcel
        
        k0 = self.greedy(p)
        self.assignment[p.id] = k0
        if k0 is None:
            # If it can fit but all channels are currently “full,” record first‐pass failure
            # and schedule a recirculation below.
            self.first_pass_failures.add(p.id)

        # Update the sliding window with the current time and parcel
        self.window.append((evt.time, p))
        while self.window and (evt.time - self.window[0][0] > self.WINDOW_DURATION):
            self.window.popleft()

        self.rebal_ctr += 1
        if self.rebal_ctr >= self.REBALANCE_INTERVAL:
            self.run_local_search()
            self.rebal_ctr = 0

        # If we have a feasible outfeed, schedule the parcel to enter it
        t = evt.time
        if k0 is None:
            # It can physically fit on some outfeed eventually, but none are open right now.
            self.recirculated_count += 1
            dt = (self.d_sc_of + self.d_between * self.num_outfeeds) / self.belt_speed
            fes.add(Event(Event.RECIRCULATE, t + dt, p))
        else:
            # Schedule the time it takes to travel from scanner to the chosen outfeed gate
            dt = (self.d_sc_of + k0 * self.d_between) / self.belt_speed
            fes.add(Event(Event.ENTER_OUTFEED, t + dt, p, outfeed_id=k0))

    def simulate(self, parcels):
        fes = FES()
        t0 = parcels[0].arrival_time

        # Seed all ARRIVAL events (times relative to t0)
        for p in parcels:
            delta = (p.arrival_time - t0).total_seconds()
            fes.add(Event(Event.ARRIVAL, delta, p))

        # Main event loop
        while not fes.isEmpty():
            e = fes.next()

            if e.type == Event.ARRIVAL:
                # Parcel travels from inlet to scanner
                dt = self.d_in_sc / self.belt_speed
                fes.add(Event(Event.ENTER_SCANNER, e.time + dt, e.parcel))

            elif e.type == Event.ENTER_SCANNER:
                # Parcel arrives at scanner; pick an outfeed or recirculate
                self.handle_enter_scanner(e, fes)

            elif e.type == Event.ENTER_OUTFEED:
                k, p = e.outfeed_id, e.parcel
                f = self.outfeeds[k]
                st = f.add_parcel(p)
                self.service_times[p.id] = st
                self.outfeed_counts[k] += 1
                self.loads[k] += st

                #Only if this parcel is now the first in the queue
                if len(f.queue) == 1:
                    # plan for the exit of this parcel
                    fes.add(Event(Event.EXIT_OUTFEED, e.time + st, p, outfeed_id=k))

                actual_enter = t0 + timedelta(seconds=e.time)
                print(
                    f"[{actual_enter.time()}] "
                    f"Parcel {p.id} ENTERS outfeed {k} with service_time={st:.3f}s"
                )

            elif e.type == Event.EXIT_OUTFEED:
                k, p = e.outfeed_id, e.parcel
                f = self.outfeeds[k]
                f.update(f.next_time)
                self.loads[k] -= self.service_times.pop(p.id)

                actual_time = t0 + timedelta(seconds=e.time)
                print(f"[{actual_time.time()}] Parcel {p.id} removed from outfeed {k}")
                if f.queue:
                # plan next exit if queue is not empty
                    fes.add(Event(Event.EXIT_OUTFEED, e.time + f.queue[0][1], f.queue[0][0], outfeed_id=k))
            elif e.type == Event.RECIRCULATE:
                # Parcel travels from the outfeed end all the way back to the scanner
                dt = (self.d_of_in + self.d_in_sc) / self.belt_speed
                fes.add(Event(Event.ENTER_SCANNER, e.time + dt, e.parcel))
        # stats
        total=len(parcels); sorted_total=sum(self.outfeed_counts)
        success=(total-len(self.first_pass_failures))/total*100
        print(f"\nTotal parcels: {total}")
        print(f"Recirculated: {self.recirculated_count}")
        print(f"Success rate: {success:.2f}%")
        for i,c in enumerate(self.outfeed_counts):
            pct=c/sorted_total*100 if sorted_total else 0
            print(f"Outfeed {i}: {c} parcels, {pct:.2f}%")
        print(f"Throughput: {sorted_total+self.recirculated_count}")

if __name__=="__main__":
    xls=pd.ExcelFile("PosiSorterData2(1).xlsx")
    df_p=clean_parcel_data(xls.parse("Parcels"))
    df_l=xls.parse("Layout")
    parcels, num_outfeeds =load_parcels_from_clean_df(df_p)
    system=PosiSorterSystem(df_l, num_outfeeds)
    system.simulate(parcels)


[09:00:15.615385] Parcel 277 ENTERS outfeed 0 with service_time=8.500s
[09:00:16.481385] Parcel 2 ENTERS outfeed 0 with service_time=8.500s
[09:00:17.171846] Parcel 1 ENTERS outfeed 1 with service_time=8.500s
[09:00:18.943846] Parcel 3 ENTERS outfeed 1 with service_time=6.500s
[09:00:20.012846] Parcel 4 ENTERS outfeed 1 with service_time=8.500s
[09:00:20.230769] Parcel 1926 ENTERS outfeed 3 with service_time=6.500s
[09:00:21.059846] Parcel 5 ENTERS outfeed 1 with service_time=8.500s
[09:00:23.307692] Parcel 1585 ENTERS outfeed 5 with service_time=7.500s
[09:00:24.115385] Parcel 277 removed from outfeed 0
[09:00:25.220308] Parcel 8 ENTERS outfeed 2 with service_time=7.500s
[09:00:25.671846] Parcel 1 removed from outfeed 1
[09:00:26.099308] Parcel 9 ENTERS outfeed 2 with service_time=8.500s
[09:00:26.730769] Parcel 1926 removed from outfeed 3
[09:00:27.400231] Parcel 7 ENTERS outfeed 4 with service_time=7.500s
[09:00:28.135308] Parcel 11 ENTERS outfeed 2 with service_time=6.500s
[09:00:2